In [ ]:
import os
import time
import argparse

import torchvision
import torch
import torch.nn as nn

from util import AverageMeter
from encoder import SmallAlexNet
from align_uniform import align_loss, uniform_loss_prelog
from tqdm import tqdm
from collections import defaultdict
import copy

import matplotlib.pyplot as plt


class TwoAugUnsupervisedDatasetLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, transform, lblmap=None):
        self.dataset = dataset
        self.transform = transform
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return self.transform(image), self.transform(image), lbl2return

    def __len__(self):
        return len(self.dataset)

def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--align_w', type=float, default=1, help='Alignment loss weight')
    parser.add_argument('--unif_w', type=float, default=1, help='Uniformity loss weight')
    parser.add_argument('--align_alpha', type=float, default=2, help='alpha in alignment loss')
    parser.add_argument('--unif_t', type=float, default=2, help='t in uniformity loss')

    parser.add_argument('--batch_size', type=int, default=256, help='Batch size')
    parser.add_argument('--epochs', type=int, default=400, help='Number of training epochs')
    parser.add_argument('--iter', type=int, default=0, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=None,
                        help='Learning rate. Default is linear scaling 0.12 per 256 batch size')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', default=[155, 170, 185], nargs='*', type=int,
                        help='When to decay learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='SGD momentum')
    parser.add_argument('--weight_decay', type=float, default=1e-4, help='L2 weight decay')
    parser.add_argument('--feat_dim', type=int, default=128, help='Feature dimensionality')

    parser.add_argument('--num_workers', type=int, default=4, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')
    parser.add_argument('--result_folder', type=str, default='./results', help='Base directory to save model')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpus = list(map(lambda x: torch.device('cuda', x), opt.gpus))

    opt.save_folder = os.path.join(
        opt.result_folder,
        f"cifar100_{opt.epochs}_4_sideinformation_align{opt.align_w:g}alpha{opt.align_alpha:g}_unif{opt.unif_w:g}t{opt.unif_t:g}_iter{opt.iter}"
    )
    os.makedirs(opt.save_folder, exist_ok=True)

    return opt


opt = parse_option()

In [ ]:
transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])

old_lbls = list(range(10))
labels_2_keep = [0,1]
# labels_2_keep = [0,1,2,3]

old2new = {}
count = 0
for old_lbl in old_lbls:
    if old_lbl in labels_2_keep: 
        old2new[old_lbl] = count
        count += 1

for old_lbl in old_lbls:
    if old_lbl not in labels_2_keep: 
        old2new[old_lbl] = count

new_lbls = list(range(count+1))

In [11]:
def get_data_loader(opt):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    dataset = TwoAugUnsupervisedDatasetLbl(
        torchvision.datasets.STL10(opt.data_folder, 'train', download=True), 
        transform=transform, 
        lblmap=old2new )
    
    return torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size, num_workers=opt.num_workers,
                                       shuffle=True, pin_memory=True)


print(f'Optimize: {opt.align_w:g} * loss_align(alpha={opt.align_alpha:g}) + {opt.unif_w:g} * loss_uniform(t={opt.unif_t:g})')

torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])

optim = torch.optim.Adam(encoder.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate,
                                                 milestones=opt.lr_decay_epochs)

loader = get_data_loader(opt)
align_meter = AverageMeter('align_loss')
unif_meter = AverageMeter('uniform_loss')
loss_meter = AverageMeter('total_loss')
it_time_meter = AverageMeter('iter_time')

for epoch in range(opt.epochs):
    align_meter.reset()
    unif_meter.reset()
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (im_x, im_y, lbl) in enumerate(loader):
        optim.zero_grad()
        x, y = encoder(torch.cat([im_x.to(opt.gpus[0]), im_y.to(opt.gpus[0])])).chunk(2)
        
        align_loss_val = align_loss(x, y, alpha=opt.align_alpha)
        # group according to new_lbls

        z = torch.cat( [x, y])
        lbl_z = torch.cat([lbl, lbl])
        unif_losses = torch.cat([uniform_loss_prelog(z[lbl_z==new_lbl]) for new_lbl in new_lbls])
        unif_loss_val = torch.log( torch.mean(unif_losses) )
        
        loss = align_loss_val * opt.align_w + unif_loss_val * opt.unif_w
        align_meter.update(align_loss_val, x.shape[0])
        unif_meter.update(unif_loss_val)
        loss_meter.update(loss, x.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(loader)}\t" +
                  f"{align_meter}\t{unif_meter}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()
    scheduler.step()

ckpt_file = os.path.join(opt.save_folder, 'encoder.pth')
torch.save(encoder.state_dict(), ckpt_file)
print(f'Saved to {ckpt_file}')

Optimize: 1 * loss_align(alpha=2) + 1 * loss_uniform(t=2)
Files already downloaded and verified
Epoch 0/400	It 0/20	align_loss 1.298860 (1.298860)	uniform_loss -2.720523 (-2.720523)	total_loss -1.421663 (-1.421663)	iter_time 2.359052 (2.359052)
Epoch 1/400	It 0/20	align_loss 0.323790 (0.323790)	uniform_loss -0.629418 (-0.629418)	total_loss -0.305628 (-0.305628)	iter_time 0.556997 (0.556997)
Epoch 2/400	It 0/20	align_loss 0.487992 (0.487992)	uniform_loss -1.008581 (-1.008581)	total_loss -0.520589 (-0.520589)	iter_time 0.594481 (0.594481)
Epoch 3/400	It 0/20	align_loss 0.610227 (0.610227)	uniform_loss -1.310382 (-1.310382)	total_loss -0.700156 (-0.700156)	iter_time 0.594522 (0.594522)
Epoch 4/400	It 0/20	align_loss 0.673948 (0.673948)	uniform_loss -1.651645 (-1.651645)	total_loss -0.977697 (-0.977697)	iter_time 0.570518 (0.570518)
Epoch 5/400	It 0/20	align_loss 0.702533 (0.702533)	uniform_loss -1.835233 (-1.835233)	total_loss -1.132699 (-1.132699)	iter_time 0.576284 (0.576284)
Epoch 6/40

In [25]:
"""
    Here we  do the linear evaluation, the old labels are provided to the linear objective as one hot
"""
import time
import argparse

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from util import AverageMeter
from encoder import SmallAlexNet
MODEL_2_LOAD = "./results/base_200_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD = "./results/base_200_4_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/manual_labels_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/base_cifar100_400_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/cifar100_400_4_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/base_cifar100_400_resize_transform_align1alpha2_unif1t2_iter0/encoder.pth"

USE_MOD_LBL = True
encoder = SmallAlexNet(feat_dim=opt.feat_dim,cifar=True).to(opt.gpus[0])
encoder.load_state_dict(torch.load(MODEL_2_LOAD))

<All keys matched successfully>

In [26]:
def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--encoder_checkpoint', type=str, help='Encoder checkpoint to evaluate', default=MODEL_2_LOAD)
    parser.add_argument('--feat_dim', type=int, default=128, help='Encoder feature dimensionality')
    parser.add_argument('--layer_index', type=int, default=-2, help='Evaluation layer')

    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--lr_decay_rate', type=float, default=0.2, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='When to decay learning rate')

    parser.add_argument('--num_workers', type=int, default=6, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpu = torch.device('cuda', opt.gpus[0])
    opt.lr_decay_epochs = list(map(int, opt.lr_decay_epochs.split(',')))

    return opt


class DatasetModifiedLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap=None):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return image, lbl2return

    def __len__(self):
        return len(self.dataset)

In [27]:
class DatasetModifiedLblandLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        return image, self.lblmap[lbl], lbl

    def __len__(self):
        return len(self.dataset)

In [28]:
def get_data_loaders(opt):
    train_transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
        torchvision.transforms.RandomHorizontalFlip()
    ])
    val_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(70),
        torchvision.transforms.CenterCrop(32),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    train_dataset = DatasetModifiedLblandLbl( torchvision.datasets.STL10(opt.data_folder, 'train', download=True, transform=train_transform), lblmap=old2new)
    val_dataset =  DatasetModifiedLblandLbl( torchvision.datasets.STL10(opt.data_folder, 'test', transform=val_transform), lblmap=old2new)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
                                               num_workers=opt.num_workers, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=opt.batch_size,
                                             num_workers=opt.num_workers, pin_memory=True)
    return train_loader, val_loader


def validate_comb(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            pred = classifier(torch.cat( (encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1), torch.nn.functional.one_hot(labels_mod.to(opt.gpus[0]), num_classes=len(labels_2_keep)+1)), dim=1)).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

def validate(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            pred = classifier( encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1) ).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

In [29]:
opt.gpu=opt.gpus[0]

In [30]:
opt = parse_option()

opt.gpu=opt.gpus[0]
torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

encoder.eval()
train_loader, val_loader = get_data_loaders(opt)

with torch.no_grad():
    sample, _ = train_loader.dataset.dataset[0]
    eval_numel = encoder(sample.unsqueeze(0).to(opt.gpus[0]), layer_index=opt.layer_index).numel()
print(f'Feature dimension: {eval_numel}')


classifier = nn.Linear(eval_numel, 10).to(opt.gpus[0]) if not USE_MOD_LBL else nn.Linear( eval_numel + len(labels_2_keep) + 1,10).to(opt.gpus[0])

optim = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma=opt.lr_decay_rate,
                                                 milestones=opt.lr_decay_epochs)

val_accs = []
loss_meter = AverageMeter('loss')
it_time_meter = AverageMeter('iter_time')
for epoch in tqdm(range(opt.epochs)):
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (images, labels_mod, labels) in enumerate(train_loader):
        optim.zero_grad()
        with torch.no_grad():
            feats = encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1)

        if USE_MOD_LBL:
            logits = classifier(torch.cat( (feats, torch.nn.functional.one_hot(labels_mod.to(opt.gpus[0]), num_classes=len(labels_2_keep)+1 )),dim=1))
        else:
            logits = classifier(feats)
        
        loss = F.cross_entropy(logits, labels.to(opt.gpus[0]))
        loss_meter.update(loss, images.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(train_loader)}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()
    scheduler.step()
    val_acc = validate_comb(opt,encoder,classifier,val_loader) if USE_MOD_LBL else validate(opt, encoder, classifier, val_loader) 
    val_accs.append(val_acc)
    print(f"Epoch {epoch}/{opt.epochs}\tval_acc {val_acc*100:.4g}%")
print(f"Best validation accuracy {max(val_accs)}")

Files already downloaded and verified
Feature dimension: 4096


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0/100	It 0/40	loss 2.317669 (2.317669)	iter_time 0.643054 (0.643054)


  1%|          | 1/100 [00:01<02:55,  1.77s/it]

Epoch 0/100	val_acc 51.59%
Epoch 1/100	It 0/40	loss 1.347377 (1.347377)	iter_time 0.140231 (0.140231)


...

Epoch 97/100	val_acc 67.42%
Epoch 98/100	It 0/40	loss 0.446620 (0.446620)	iter_time 0.173109 (0.173109)


 99%|█████████▉| 99/100 [02:07<00:01,  1.27s/it]

Epoch 98/100	val_acc 67.49%
Epoch 99/100	It 0/40	loss 0.483180 (0.483180)	iter_time 0.170703 (0.170703)


100%|██████████| 100/100 [02:08<00:00,  1.28s/it]

Epoch 99/100	val_acc 67.46%
Best validation accuracy 0.675625
